In [1]:
import numpy as np
import math

## 単体の関数

### 1. 均時差と赤緯 - 赤坂の式

参考：拡張アメダス気象データ1981-2000解説書 8.1　太陽位置の計算

#### Outline

$$
\displaystyle
n = Y - 1968
$$

$$
\displaystyle
d_0 = 3.71 + 0.2596 \cdot n - INT \biggl[ \frac{ n + 3 }{ 4 } \biggr]
$$

$$
\displaystyle
M = \frac{ 360 \cdot (D - d_0) }{ D_{ay} }
$$

$$
\displaystyle
\epsilon = 12.3901 + 0.0172 \cdot \biggl( n + \frac{ M }{ 360 } \biggr)
$$

$$
\displaystyle
v = M + 1.914 \cdot \sin M + 0.02 \cdot \sin(2M)
$$

$$
\displaystyle
E_t = \left ( (M - v) - \tan^{-1} \left( \frac{ 0.043 \cdot \sin 2( v + \epsilon ) }{ 1 - 0.043 \cdot \cos 2(v + \epsilon) } \right) \right) \div 15
$$

$$
\displaystyle
\sin \delta = \cos ( v + \epsilon ) \cdot \sin \delta_0
$$

$INT$ 小数点以下切り捨てを表す

Input:  
$Y$ is year / 西暦年  
$D$ is total day / 年通算日(1/1を $D=1$ とする), d    

$n$ is the difference of year from 1968 / 1968年との年差  
$d_0$ is 平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）, d    
$D_{ay}$ is anomalistic year / 近点年（近日点基準の公転周期日数） = 365.2596, d    
$M$ is mean anomaly / 平均近点離角, deg  
$\epsilon$ is angle between perihelion and winter solstitial point / 近日点と冬至点の角度, deg   
$v$ is true anomaly / 真近点離角, deg  
$E_t$ is equation of time / 均時差, h  
$\delta_0$ is daily declination of winter solstice of northern hemisphere / 北半球の冬至の日赤緯 = -23.4393, deg    
$\delta$ is declination / 赤緯, rad  

#### Function

In [2]:
def get_eqation_of_time_and_declination_AKASAKA(year, nday):

    # anomalistic year / 近点年
    D_AY = 365.2596

    # daily declination of winter solstice of northern hemisphere / 北半球の冬至の日赤緯
    DLT0 = math.radians(-23.4393)
    
    # the difference of year from 1968 / 1968年との年差  
    n = year - 1968
    
    # 平均軌道上の近日点通過日（暦表時による1968年1月1日正午基準の日差）, d        
    d0 = 3.71 + 0.2596 * n - int((n + 3) / 4)    
    
    # mean anomaly / 平均近点離角, deg  
    M = 360. * (nday - d0) / D_AY
    
    # angle between perihelion and winter solstitial point / 近日点と冬至点の角度, deg   
    eps = 12.3901 + 0.0172 * (n + M / 360)
    
    # true anomaly / 真近点離角, deg  
    v = M + 1.914 * math.sin(math.radians(M)) \
        + 0.02 * math.sin(math.radians(2. * M))

    veps = math.radians(v + eps)
    
    # equation of time / 均時差, h  
    et = (M - v) \
         - math.degrees(math.atan(
                        0.043 * math.sin(2. * veps)
                        / (1.0 - 0.043 * math.cos(2.0 * veps))
                        ))
    et_hour = et / 15.0
    
    # declination / 赤緯, rad  
    sindlt = math.cos(veps) * math.sin(DLT0)
    cosdlt = (abs(1. - sindlt**2)) ** 0.5
    dlt = math.atan2(sindlt, cosdlt)
    
    return et_hour, dlt

### 2. 均時差と赤緯 - HASPの方法

#### 赤緯

$$
\begin{align}
\displaystyle
\delta_d &= 0.006322 - 0.405748 \cos (2 \pi n / 366 + 0.153231)\\
& - 0.005880 \cos (4 \pi n / 366 + 0.207099)\\
& - 0.003233 \cos (6 \pi n / 366 + 0.620129)
\end{align}
$$

$\delta$ is declination / 赤緯, rad  
$n$ is total day / 年通算日(1/1を $D=1$ とする), d    

In [3]:
def calc_deltad(NDay):
    return 0.006322 \
           - 0.405748 * math.cos(2 * math.pi * float(NDay) / 366 + 0.153231) \
           - 0.005880 * math.cos(4 * math.pi * float(NDay) / 366 + 0.207099) \
           - 0.003233 * math.cos(6 * math.pi * float(NDay) / 366 + 0.620129)

#### 均時差

$$
\begin{align}
e_d &= -0.000279 \\
& + 0.122772 \cos (2 \pi N / 366 + 1.498311) \\
& - 0.165458 \cos (4 \pi N / 366 - 1.261546) \\
& - 0.005354 \cos (6 \pi N / 366 - 1.1571)
\end{align}
$$


$e_d$ is equation of time / 均時差, h  
$N$ is total day / 年通算日(1/1を$N=1$とする), d

In [4]:
def calc_eed(NDay):
    return ( -0.000279 + 0.122772 * math.cos(2 * math.pi * NDay / 366 + 1.498311)
                       - 0.165458 * math.cos(4 * math.pi * NDay / 366 - 1.261546)
                       - 0.005354 * math.cos(6 * math.pi * NDay / 366 - 1.1571)   )

#### Function

In [5]:
def get_eqation_of_time_and_declination_HASP(nday):
    e = calc_eed(nday)
    delta = calc_deltad(nday)
    return e, delta

### 3. 時角

$$
\displaystyle
T = (t + e_d - 12) \times 15 + (L - L_0)
$$

$T$ is hour angle / 時角, rad  
$t$ is time / 時刻, h  
$e_d$ is equation of time / 均時差, h  
$L$ is longitude 経度, rad  
$L_0$ is the longitude of the location where the local time defined / 標準時の地点の経度（=135.0 degree （日本の場合））, rad

In [6]:
def calc_Tdt(longitude, eed, TT, longitude_std):
    return (TT + eed - 12) * math.radians(15.0) + longitude - longitude_std

### 4. 太陽高度

#### Outline

$$
\displaystyle
\sin h_S = \max(0, \sin \phi \sin \delta + \cos \phi \cos \delta \cos T)
$$

$$
\displaystyle
\cos h_S = (1 - \sin ^2 h_S)^{0.5}
$$

$h_S$ is solar altitude / 太陽高度, rad  
$\phi$ is latitude, 緯度, rad  
$\delta$ is declination / 赤緯, rad   
$T$ is hour angle / 時角, rad

#### Function

In [7]:
def calc_hs(latitude, delta, t):

    sin_hs = max(0.0,
               math.sin(latitude) * math.sin(delta) \
               + math.cos(latitude) * math.cos(delta) * math.cos(t)
              )
    
    cos_hs = (1 - sin_hs**2) **0.5
    
    if sin_hs == 1.0:
        hs = math.radians(90.0)
    else:
        hs = math.atan2(sin_hs, cos_hs)
    
    return hs, sin_hs, cos_hs

### 5. 太陽方位角

三浦コメント：  
分母が0の場合を回避しなくてもよいのか？  
高度が0のときでも太陽方位角は定義できるので,h>0 で始めるif文は不要では。（西澤さんの式にはなかった。児島さんの式にはあった。）  hs の計算の時点で0より大の制約をいれている以上、ここでも何らかの処理は必要かと思う。あるいは、hs の計算の時点では0より大の処理は含めず、全体計算でhs>0 の場合わけを行うか？

#### Outline

$$
\displaystyle
\sin A_{ZS} = \frac{ \cos \delta \sin T }{ \cos h_S }
$$

$$
\displaystyle
\cos A_{ZS} = \frac { \sin h_S \sin \phi - \sin \delta }{ \cos h_S \cos \phi }
$$

$A_{ZS}$ is solar azimuth / 太陽方位角, rad  
$h_S$ is solar altitude / 太陽高度, rad  
$\delta$ is declination / 赤緯, rad  
$T$ is hour angle / 時角, rad  
$\phi$ is latitude / 緯度, rad

#### Function

In [8]:
def calc_Azs(latitude, delta, t, hs):
    
    sinAzs = math.cos(delta) * math.sin(t) / math.cos(hs)
    cosAzs = (math.sin(hs) * math.sin(latitude) - math.sin(delta)) \
             / (math.cos(hs) * math.cos(latitude))
    
    if sinAzs == 1.0:
        return math.radians(90.0)
    elif sinAzs == -1.0:
        return math.radians(270.0)
    else:
        return math.atan2(sinAzs, cosAzs)

#    if hs > 0.0:
#        if sinAzs == 1.0:
#            return math.radians(90.0)
#        elif sinAzs == -1.0:
#            return math.radians(270.0)
#        else:
#            return math.atan2(sinAzs, cosAzs)
#    else:
#        return 0.0

## 複合関数

### 1. 太陽位置

#### Input

`year` is year / 西暦年  
`nday` is total day / 年通算日(1/1を $D=1$ とする), d    
`tm` is standard hour / 標準時, h  
`lat` is latitude / 計算対象地点の緯度, rad  
`lon` is longitude / 計算対象地点の経度, rad  
`lonstd` is longitude of the meridian (line) in Japan / 標準時の子午線の経度, rad

#### Output

`hs` is solar altitude / 太陽高度, rad  
`A` is solar azimuth / 太陽方位角, rad  

#### Function

In [9]:
def calc_solar_position(year, nday, tm, latitude, longitude, longitude_std, method):

    # equation time / 均時差, h & declination / 赤緯, rad
    if method == 'AKASAKA':
        et_hour, dlt = get_eqation_of_time_and_declination_AKASAKA(year, nday)
    elif method == 'HASP':
        et_hour, dlt = get_eqation_of_time_and_declination_HASP(nday)
    else:
        raise IndexError('Error: Invalid index of the medhod  of solar position.')

    # hour angle / 時角, rad
    hour_angle = calc_Tdt(longitude, et_hour, tm, longitude_std)
    
    # solar altitude / 太陽高度, rad, -, -
    hs, sin_hs, cos_hs = calc_hs(latitude, dlt, hour_angle)
    
    # solar azimuth / 太陽方位角, rad
    A = calc_Azs(latitude, dlt, hour_angle, hs)

    return hs, A

In [10]:
calc_solar_position(2011, 121, 12.5, math.radians(23.5), math.radians(140.0), math.radians(135.0), 'AKASAKA')

(1.3068121993844721, 1.0077797614951318)

## A.8 窓面の方位 (仕様書5.2 図4)

- 窓面の方位は、以下の通り
  - 北北東：$-157.5°$, 北東：$-135°$, …, 東：$-90°$, …, 南：$0°$, …, 西：$+90°$, …,北：$+180°$
  - 角度指定も可：$-180°< A_{ZW,j} \leq +180°$
  - デフォルトは8方位指定

In [11]:
def calc_Azwj(Azimuth):
    
    Azimuth00 = ["北北東", "北東", "東北東", "東", "東南東", "南東", "南南東", "南"
                 , "南南西", "南西", "西南西", "西", "西北西", "北西", "北北西", "北" ]
    if Azimuth in Azimuth00:
        return (Azimuth00.index(Azimuth) - 7) * 22.5
    elif -180 < float(Azimuth) <= 180:
        return float(Azimuth) 
    else:
        raise ValueError('窓面方位の入力が不適切です')

## A.9 窓面の法線ベクトルと太陽位置とのなす水平面上の角度の計算 (仕様書6.2 式(1))

- 窓面の法線ベクトルと太陽位置とのなす水平面上の角度$A_{ZW,j,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,j}[deg]$

$$
A_{ZW,j,d,t} = \left\{
\begin{array}{ll}
A_{ZS,d,t} - A_{ZW,j} \hspace{48pt} (-180 < A_{ZS,d,t} - A_{ZW,j} \leq 180)
\\
A_{ZS,d,t} - A_{ZW,j} + 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \leq -180)
\\
A_{ZS,d,t} - A_{ZW,j} - 360 \hspace{24pt} (A_{ZS,d,t} - A_{ZW,j} \geq 180)
\end{array}
\right.  \qquad (1) 
$$

In [12]:
def calc_Azwjdt(Azwj, Azsdt):
    
    Azwjdt = Azsdt - Azwj
    if Azwjdt < -180:
        return Azwjdt + 360.0
    elif Azwjdt > 180:
        return Azwjdt - 360.0
    else:
        return Azwjdt